# Preprocessing

- load audio
- put them into arrays
- perform cqt transform
- 

In [ ]:
import os
import librosa
import pandas as pd
import numpy as np

train_audio_dir = 'datasets/musicnet/musicnet/train_data'
train_labels_dir = 'datasets/musicnet/musicnet/train_labels'

train_audio_data = {}
train_labels_data = {}

LIMIT = 10
count = 0
for filename in os.listdir(train_audio_dir):
    if count >= LIMIT:
        break
    else:
        filepath = os.path.join(train_audio_dir, filename)
        audio_data, sr = librosa.load(filepath, sr=None)
        train_audio_data[filename] = {'audio': audio_data, 'sample_rate': sr}
        count +=1

count = 0
for filename in os.listdir(train_labels_dir):
    if count >= LIMIT:
        break
    else:
        filepath = os.path.join(train_labels_dir, filename)
        train_labels_data[filename] = pd.read_csv(filepath)
        count+=1

In [ ]:
print(train_audio_data)
print(train_labels_data)

In [ ]:
train_audio_array = []
train_label_array = []

for filename, data in train_audio_data.items():
    audio_array = data['audio']
    train_audio_array.append(audio_array)

for filename, data in train_labels_data.items():
        label_array = data.values
        train_label_array.append(label_array)


print(train_audio_array)
print(train_label_array)

In [ ]:
import librosa

train_cqt_array = [librosa.cqt(audio) for audio in train_audio_array]

In [ ]:
import matplotlib.pyplot as plt

for i in range(2):
    cqt_array = train_cqt_array[i]
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.amplitude_to_db(cqt_array, ref=np.max), sr=sr, x_axis='time', y_axis='cqt_note')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Constant-Q Transform (CQT) - Audio {}'.format(i+1))
    plt.tight_layout()
    plt.show()

In [33]:
# Padding is essential to have uniform train array to feed to neural net
# Find the maximum length among all CQT arrays
max_length = max(len(cqt_array[0]) for cqt_array in train_cqt_array)

# Pad all CQT arrays to match the maximum length
train_padded_cqt_arrays = []
for cqt_array in train_cqt_array:
    padded_cqt_array = np.pad(cqt_array, ((0, 0), (0, max_length - len(cqt_array[0]))), mode='constant')
    train_padded_cqt_arrays.append(padded_cqt_array)

# convert to numpy to check shape, if the array is not uniform it won't convert
train_padded_cqt_arrays = np.array(train_padded_cqt_arrays)
print(train_padded_cqt_arrays.shape) # number of audio files, number of frequency bin/range, number of timesteps

(10, 84, 79633)
